# Evaluate DeepDive with Auditory Perception using several training set types.

We are using Auditory Perception as it is a known MeSH term and therefore was easy to retrieve programatically from PubMed.

When documents retrieved by Auditory Perception were reviewed by annotators, they were all or nearly all identified as 'Relevant to Auditory Perception', thereby making a pretty clean test set with or without annotator oversight.

## Main questions addressed.
*Q1* Qualitatively, how substantial is the effect of increasing the distance of the negative training set?

*H1* We expect incresed distance to increase specificity of the desired class, and reduce false postives from the undesired class(es).

*Q2* Qualitatively, does increasing the desired class size vastly improve specificity and reduce false positives?

*H2* We expect more training data will dramatically improve our test measures.

## Start Server

In [1]:
!pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start # deepdive

pg_ctl: another server might be running; trying to start server anyway
server starting


## experiments we are running

In [2]:
import os
import shutil
import errno
import glob

dd_sent_sources = '/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/task_data_sentences'
dd_app_dir = '/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app'
templates = '/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/templates_deepdive_app_bagofwords/'

conf_matrix_r_src = '/Users/ccarey/Documents/Projects/NAMI/rdoc/scripts/report_dd_confusion_matrix.R' # creates tsv and pdf reports.


In [33]:
def create_dd_app(template_dir, app_name, input_raw, input_annotated):
    '''Sets up a DeepDive app based and populates the input data'''
    try:
        shutil.copytree(template_dir, app_name)
    except OSError as err:
        print("Error copying {} to {}: {}".format(template_dir, app_name, err))
    # create / overwrite unique postgres db name for this app:
    with open(os.path.join(app_name, 'db.url'), 'w') as f:
        f.write('postgresql://localhost/{}\n'.format(app_name))
    try:
        shutil.copyfile(input_raw, os.path.join(app_name, 'input', 'raw_sentences'))
    except OSError as err:
        print("Error copying raw sentences: {}".format(err))
    try:
        shutil.copyfile(input_annotated, os.path.join(app_name, 'input', 'annotated_sentences'))
    except OSError as err:
        print("Error copying annotated sentences: {}".format(err))

def my_dd_table_sql_str():
    '''Used to populate cc_all_predictions.tsv or such table in 
    deepdive app.
    
    That tsv file in turn is used by our confusion matrix R Script to
    generate stats and plots from our deepdive apps.
    '''
    cmd = ('SELECT a.has_term, r.terms, a.sentence_id, expectation '
           'FROM '
           '_annotated_sentences_has_term_inference as a JOIN '
           '_raw_sentences as r ON '
           'a.sentence_id = r.sentence_id '
           'ORDER BY a.sentence_id')
    return(cmd)


## Q1. experiments: smaller 'true' training set, different distances of 'false' training sets.

### Create deepdive apps for Q1 named according to their input data.

In [20]:
# all apps must be at depth = 1, in a deepdive_app home directory, in our case 'deepdive_app/'
print(dd_app_dir)
%cd $dd_app_dir

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app


In [21]:
sources_q1 = !ls -d {dd_sent_sources}/Auditory_146*

for s in sources_q1:
    app_name = os.path.basename(s)
    print('Creating deepdive_app {} at : {}'.format(app_name, os.getcwd()))
    create_dd_app(template_dir=templates, 
                  app_name=app_name,
                  input_raw=os.path.join(s, 'raw_sentences'),
                  input_annotated=os.path.join(s, 'annotated_sentences'))

Creating deepdive_app Auditory_146__vs__Arousal_1_1000__pdx__un_Aro_154 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Creating deepdive_app Auditory_146__vs__Arousal_1_1000__pdx__un_Aud_1_1000 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Creating deepdive_app Auditory_146__vs__Disease_1000__pdx__un_Aud_1_1000 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Creating deepdive_app Auditory_146__vs__Psyc_1000__pdx__un_Aud_1_1000 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app


## run deepdive apps for Q1.
The app must be initialized and run within the apps directory.

In [22]:
%cd {dd_app_dir}

apps = glob.glob('Auditory_146__vs*')
apps = [os.path.basename(a) for a in apps]

for app in apps:
    print('Running deepdive_app {}'.format(app))
    %cd {app}
    !deepdive initdb 1> /dev/null 2>&1
    !deepdive run  1> /dev/null 2>&1
    %cd {dd_app_dir}

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_146__vs__Arousal_1_1000__pdx__un_Aro_154
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Arousal_1_1000__pdx__un_Aro_154
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_146__vs__Arousal_1_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Arousal_1_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_146__vs__Disease_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Disease_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_146__vs__Psyc_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Psyc_1000__pdx__un_Aud_1_1000
/Users/ccarey/Do

## collate tables and stats for Q1 deepdive apps.

In [25]:
cmd = my_dd_table_sql_str()

%cd {dd_app_dir}
for a in apps:
    %cd {a}
    !deepdive sql eval "{cmd}" format=tsv > cc_all_predictions.tsv
    !RScript {conf_matrix_r_src} ./cc_all_predictions.tsv {a} '' > /dev/null 2>&1
    %cd {dd_app_dir}

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Arousal_1_1000__pdx__un_Aro_154
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Arousal_1_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Disease_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_146__vs__Psyc_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app


## Q2. experiments: larger 'true' training set, different distances of 'false' training sets.

Although we are working with approximately 2x as many abstracts, remains compute time is quite fast.

In [28]:
# all apps must be at depth = 1, in a deepdive_app home directory, in our case 'deepdive_app/'
print(dd_app_dir)
%cd $dd_app_dir

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app


In [30]:
sources_q1 = !ls -d {dd_sent_sources}/Auditory_2_1000*

for s in sources_q1:
    app_name = os.path.basename(s)
    print('Creating deepdive_app {} at : {}'.format(app_name, os.getcwd()))
    create_dd_app(template_dir=templates, 
                  app_name=app_name,
                  input_raw=os.path.join(s, 'raw_sentences'),
                  input_annotated=os.path.join(s, 'annotated_sentences'))

Creating deepdive_app Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aro_154 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Creating deepdive_app Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aud_1_1000 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Creating deepdive_app Auditory_2_1000__vs__Disease_1000__pdx__un_Aud_1_1000 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Creating deepdive_app Auditory_2_1000__vs__Psyc_1000__pdx__un_Aud_1_1000 at : /Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app


In [31]:
%cd {dd_app_dir}

apps = glob.glob('Auditory_2_1000__vs*')
apps = [os.path.basename(a) for a in apps]

for app in apps:
    print('Running deepdive_app {}'.format(app))
    %cd {app}
    !deepdive initdb 1> /dev/null 2>&1
    !deepdive run  1> /dev/null 2>&1
    %cd {dd_app_dir}

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aro_154
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aro_154
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_2_1000__vs__Disease_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Disease_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
Running deepdive_app Auditory_2_1000__vs__Psyc_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Psyc_1000__pdx__un_Aud

In [32]:
cmd = my_dd_table_sql_str()

%cd {dd_app_dir}
for a in apps:
    %cd {a}
    !deepdive sql eval "{cmd}" format=tsv > cc_all_predictions.tsv
    !RScript {conf_matrix_r_src} ./cc_all_predictions.tsv {a} '' > /dev/null 2>&1
    %cd {dd_app_dir}

/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aro_154
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Arousal_1_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Disease_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_2_1000__vs__Psyc_1000__pdx__un_Aud_1_1000
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app


# Appendix: reporting stats

for f in  Aud*; do echo -e "\n\n\n${f}\n"; cat ${f}/*test_only__stats.tsv; done | sed 's/[.]\(...\).*/.\1/'

for f in  Aud*; do echo -e "\n\n\n${f}\n"; cat ${f}/*test_only__confmatr.tsv; done

Auditory_Perception__vs__Arousal_un_Aro156

                    KNOWN
             "Negative"	"Positive"
     PRED "Negative"	256	8
          "Positive"	5	33

- "Accuracy"	0.956
- "AccuracyLower"	0.927
- "AccuracyUpper"	0.976
- "Sensitivity"	0.804
- "Specificity"	0.980
- "Pos Pred Value"	0.868
- "Neg Pred Value"	0.969
- "Prevalence"	0.135


Auditory_Perception__vs__Arousal_un_Aud

                    KNOWN
             "Negative"	"Positive"
    PRED "Negative"	242	13
         "Positive"	3	25

- "Accuracy"	0.943
- "AccuracyLower"	0.909
- "AccuracyUpper"	0.967
- "Sensitivity"	0.657
- "Specificity"	0.987
- "Pos Pred Value"	0.892
- "Neg Pred Value"	0.949
- "Prevalence"	0.134


Auditory_Perception__vs__Disease_un_Aud

           "Negative"	"Positive"
      "Negative"	233	1
      "Positive"	2	32

- "Accuracy"	0.988
- "AccuracyLower"	0.967
- "AccuracyUpper"	0.997
- "Sensitivity"	0.969
- "Specificity"	0.991
- "Pos Pred Value"	0.941
- "Neg Pred Value"	0.995
- "Prevalence"	0.123


Auditory_Perception__vs__Psyc_un_Aud

            "Negative"	"Positive"
        "Negative"	249	6
        "Positive"	4	34

- "Accuracy"	0.965
- "AccuracyLower"	0.938
- "AccuracyUpper"	0.983
- "Sensitivity"	0.85
- "Specificity"	0.984
- "Pos Pred Value"	0.894
- "Neg Pred Value"	0.976
- "Prevalence"	0.136


# Appendix
## A.1 DeepDive crashes when run below directory of deepdive_app during GIBBS sampling

Note failure below.

But runs fine if we move the app to:
/Users/ccarey/Documents/Projects/NAMI/rdoc/tasks/deepdive_app/Auditory_Perception__vs__Disease_un_Aud

### This might be related to another error at the same place I encountered where I had long path names.

The GIBBS SAMPLING is failing at either app_name is > 55 characters OR Full path + app_name > 117 characters.

## A.2 DeepDive crashes when sentences are empty
Possibly an issue with my UDF.

# TODO: See if fix to udf corrects this crash?
For example, we had following records that were empty.

Removing these fixed the problem.

Error generated was probably within my udf script as indicated: